In [7]:
import os
import sys
import dask
import zarr
import xesmf as xe
import numpy as np
import xarray as xr
from glob import glob

In [8]:
import matplotlib.pyplot as plt
%matplotlib inline

In [9]:
import time

## Variable info

In [10]:
upper_dir = '/glade/campaign/cisl/aiml/ksha/CREDIT_data/ERA5_plevel_base/upper_air/'
surf_dir = '/glade/campaign/cisl/aiml/ksha/CREDIT_data/ERA5_plevel_base/surf/'

In [11]:
# ds_404 = xr.open_zarr('/glade/campaign/ral/hap/ksha/DWC_data/CONUS_domain_GP/raw_404/C404_GP_1979-10-01_00H.zarr')
# P = ds_404['WRF_P'].values
# P.mean(axis=(-2, -1))[0, :] / 100

In [12]:
level_pick = [8, 10, 14, 17, 19, 21, 23, 25, 30, 34, 36]
varname_surf = ['MSL', 'SP', 'VAR_10U', 'VAR_10V', 'VAR_2T']

In [19]:
for i_year, year in enumerate(range(1979, 2024)[:1]):
    ds_upper = xr.open_zarr(upper_dir + f'ERA5_plevel_6h_upper_air_{year}.zarr')
    ds_surf = xr.open_zarr(surf_dir + f'ERA5_plevel_6h_surf_{year}.zarr')
    
    ds_upper = ds_upper.isel(
        level=level_pick,
        latitude=slice(180, 268),
        longitude=slice(1001, 1113)
    ).drop_vars(['W'])
    
    ds_surf = ds_surf[varname_surf].isel(
        latitude=slice(180, 268),
        longitude=slice(1001, 1113)
    )
    
    ds_final = xr.merge([ds_upper, ds_surf])
    # surface --> top, south --> north | align with WRF data 
    ds_final = ds_final.isel(level=slice(None, None, -1), latitude=slice(None, None, -1))
    ds_final = ds_final.load()
    
    if i_year == 0:
        varnames = list(ds_final.keys())
        # zarr encodings
        dict_encoding = {}
        
        chunk_size_3d = dict(chunks=(1, 88, 112))
        chunk_size_4d = dict(chunks=(1, 11, 88, 112))
        
        compress = zarr.Blosc(cname='zstd', clevel=1, shuffle=zarr.Blosc.SHUFFLE, blocksize=0)
        
        for i_var, var in enumerate(varnames):
            if var in varname_surf:
                dict_encoding[var] = {'compressor': compress, **chunk_size_3d}
            else:
                dict_encoding[var] = {'compressor': compress, **chunk_size_4d}
    
    save_name = f'/glade/derecho/scratch/ksha/DWC_data/CONUS_domain_GP/all_in_one/ERA5_GP_{year}.zarr'
    
    #ds_final.to_zarr(save_name, mode='w', consolidated=True, compute=True)
    print(save_name)

In [23]:
xr.open_zarr('/glade/derecho/scratch/ksha/DWC_data/CONUS_domain_GP/all_in_one/ERA5_GP_1979.zarr')

<xarray.Dataset>
Dimensions:    (time: 1460, latitude: 88, longitude: 112, level: 11)
Coordinates:
  * latitude   (latitude) float64 45.0 44.75 44.5 44.25 ... 23.75 23.5 23.25
  * level      (level) float64 1e+03 950.0 850.0 700.0 ... 200.0 100.0 50.0
  * longitude  (longitude) float64 250.2 250.5 250.8 251.0 ... 277.5 277.8 278.0
  * time       (time) datetime64[ns] 1979-01-01 ... 1979-12-31T18:00:00
Data variables:
    MSL        (time, latitude, longitude) float32 dask.array<chunksize=(1, 88, 112), meta=np.ndarray>
    Q          (time, level, latitude, longitude) float32 dask.array<chunksize=(1, 11, 88, 112), meta=np.ndarray>
    SP         (time, latitude, longitude) float32 dask.array<chunksize=(1, 88, 112), meta=np.ndarray>
    T          (time, level, latitude, longitude) float32 dask.array<chunksize=(1, 11, 88, 112), meta=np.ndarray>
    U          (time, level, latitude, longitude) float32 dask.array<chunksize=(1, 11, 88, 112), meta=np.ndarray>
    V          (time, level, latitude, longitude) float32 dask.array<chunksize=(1, 11, 88, 112), meta=np.ndarray>
    VAR_10U    (time, latitude, longitude) float32 dask.array<chunksize=(1, 88, 112), meta=np.ndarray>
    VAR_10V    (time, latitude, longitude) float32 dask.array<chunksize=(1, 88, 112), meta=np.ndarray>
    VAR_2T     (time, latitude, longitude) float32 dask.array<chunksize=(1, 88, 112), meta=np.ndarray>
    Z          (time, level, latitude, longitude) float32 dask.array<chunksize=(1, 11, 88, 112), meta=np.ndarray>
Attributes:
    CONVERSION_DATE:      Mon May 13 18:11:19 MDT 2019
    CONVERSION_PLATFORM:  Linux casper13 3.10.0-693.21.1.el7.x86_64 #1 SMP We...
    Conventions:          CF-1.6
    DATA_SOURCE:          ECMWF: https://cds.climate.copernicus.eu, Copernicu...
    NCO:                  netCDF Operators version 4.7.4 (http://nco.sf.net)
    NETCDF_COMPRESSION:   NCO: Precision-preserving compression to netCDF4/HD...
    NETCDF_CONVERSION:    CISL RDA: Conversion from ECMWF GRIB 1 data to netC...
    NETCDF_VERSION:       4.6.1
    history:              Mon May 13 18:11:48 2019: ncks -4 --ppc default=7 e...

In [20]:
ds_final

<xarray.Dataset>
Dimensions:    (time: 1460, level: 11, latitude: 88, longitude: 112)
Coordinates:
  * latitude   (latitude) float64 45.0 44.75 44.5 44.25 ... 23.75 23.5 23.25
  * level      (level) float64 1e+03 950.0 850.0 700.0 ... 200.0 100.0 50.0
  * longitude  (longitude) float64 250.2 250.5 250.8 251.0 ... 277.5 277.8 278.0
  * time       (time) datetime64[ns] 1979-01-01 ... 1979-12-31T18:00:00
Data variables:
    Q          (time, level, latitude, longitude) float32 0.0002775 ... 2.806...
    T          (time, level, latitude, longitude) float32 259.8 260.9 ... 208.5
    U          (time, level, latitude, longitude) float32 1.485 1.934 ... 7.02
    V          (time, level, latitude, longitude) float32 0.02861 ... -0.08144
    Z          (time, level, latitude, longitude) float32 2.444e+03 ... 2.012...
    MSL        (time, latitude, longitude) float32 1.037e+05 ... 1.015e+05
    SP         (time, latitude, longitude) float32 7.208e+04 ... 1.012e+05
    VAR_10U    (time, latitude, longitude) float32 2.224 1.555 ... 5.084 4.75
    VAR_10V    (time, latitude, longitude) float32 0.09012 0.1272 ... -3.28
    VAR_2T     (time, latitude, longitude) float32 240.6 242.5 ... 295.7 295.7
Attributes:
    CONVERSION_DATE:      Mon May 13 18:11:19 MDT 2019
    CONVERSION_PLATFORM:  Linux casper13 3.10.0-693.21.1.el7.x86_64 #1 SMP We...
    Conventions:          CF-1.6
    DATA_SOURCE:          ECMWF: https://cds.climate.copernicus.eu, Copernicu...
    NCO:                  netCDF Operators version 4.7.4 (http://nco.sf.net)
    NETCDF_COMPRESSION:   NCO: Precision-preserving compression to netCDF4/HD...
    NETCDF_CONVERSION:    CISL RDA: Conversion from ECMWF GRIB 1 data to netC...
    NETCDF_VERSION:       4.6.1
    history:              Mon May 13 18:11:48 2019: ncks -4 --ppc default=7 e...